In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable
from torch.nn.parameter import Parameter

import import_ipynb
import math
import utils
import argparse
import data_loader
import numpy as np
import pandas as pd

from ipdb import set_trace
from sklearn import metrics


class FeatureRegression(nn.Module):
    def __init__(self, input_size):
        super(FeatureRegression, self).__init__()
        self.build(input_size)
        

    def build(self, input_size):
        self.W = Parameter(torch.Tensor(input_size, input_size))
        self.b = Parameter(torch.Tensor(input_size))

        m = torch.ones(input_size, input_size) - torch.eye(input_size, input_size)
        self.register_buffer('m', m)
        
        self.reset_parameters()

    def reset_parameters(self):
        dw = pd.read_csv("E:/데이터모음/대기오염/preprocess/latlong_distance.csv") # this data is for using IDW. I already calulate the distance based on latitude and logitude of monitoing sites.
        self.W.data=torch.from_numpy(dw.values).float()
        stdv = 1. / math.sqrt(self.W.size(0))
#         self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, x):
        z_h = F.linear(x, self.W * Variable(self.m), self.b) # W * self.m -> considering feature correlate
        return z_h

class TemporalDecay(nn.Module):
    def __init__(self, input_size, output_size, diag = False):
        super(TemporalDecay, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.diag = diag

        self.build(self.input_size, self.output_size)

    def build(self, input_size, output_size):
        self.W = Parameter(torch.Tensor(self.output_size, self.input_size))
        self.b = Parameter(torch.Tensor(self.output_size))
        
        if self.diag == True:
            assert(self.input_size == self.output_size)
            m = torch.eye(self.input_size, self.input_size)
            self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, d):
        if self.diag == True:
            gamma = F.relu(F.linear(d, self.W * Variable(self.m), self.b))
        else:
            gamma = F.relu(F.linear(d, self.W, self.b))
        gamma = torch.exp(-gamma)
        return gamma

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.build()

    def build(self):
        self.rnn_hid_size = 64
        self.input_size = 25
        self.rnn_cell = nn.RNNCell(self.input_size * 2, self.rnn_hid_size)

        self.temp_decay_h = TemporalDecay(input_size = self.input_size, output_size = self.rnn_hid_size, diag = False) # input_size : variable number?
        self.temp_decay_x = TemporalDecay(input_size = self.input_size, output_size = 25, diag = True)
        self.hist_reg = nn.Linear(self.rnn_hid_size,25)
        self.feat_reg = FeatureRegression(25)

        self.weight_combine = nn.Linear(self.input_size * 2, 25)

        self.dropout = nn.Dropout(p = 0.25)
        self.out = nn.Linear(self.rnn_hid_size, 1)

    def forward(self, data, direct, seq_len):
        # Original sequence with 24 time steps
        self.seq_len = seq_len
        values = data[direct]['values']
        masks = data[direct]['masks']
        deltas = data[direct]['deltas']

        evals = data[direct]['evals']
        eval_masks = data[direct]['eval_masks']

        h = Variable(torch.zeros((values.size()[0], self.rnn_hid_size)))
        c = Variable(torch.zeros((values.size()[0], self.rnn_hid_size)))

        if torch.cuda.is_available():
            h, c = h.cuda(), c.cuda()

        loss = 0.0

        imputations = []
        
        for t in range(self.seq_len):
            x = values[:, t, :]  # (batch_size, seq_len, input_size=output) : (batch_size,120,25) -> (batch_size,25)
            m = masks[:, t, :]   # (batch_size, length of time, length of features)
            d = deltas[:, t, :]

            gamma_h = self.temp_decay_h(d)
            gamma_x = self.temp_decay_x(d)

#             h = h * gamma_h
            x_h = self.hist_reg(h)
            loss += torch.sum(torch.abs(x - x_h) * m) / (torch.sum(m) + 1e-5)

            x_c =  m * x +  (1 - m) * x_h
            z_h = self.feat_reg(x_c) # feature-based estimation
            loss += torch.sum(torch.abs(x - z_h) * m) / (torch.sum(m) + 1e-5)
            
            beta = self.weight_combine(torch.cat([gamma_x, m], dim = 1))
            c_h = beta * z_h + (1 - beta) * x_h
            loss += torch.sum(torch.abs(x - c_h) * m) / (torch.sum(m) + 1e-5)
            c_c = m * x + (1 - m) * c_h
            h = h * gamma_h
            
            inputs = torch.cat([c_c, m], dim = 1)
            
            h = self.rnn_cell(inputs, h)
            imputations.append(c_c.unsqueeze(dim = 1))

        imputations = torch.cat(imputations, dim = 1)

        return {'loss': loss/self.seq_len, 'imputations': imputations, 'evals': evals, 'eval_masks': eval_masks}

    def run_on_batch(self, data, optimizer, seq_len):
        ret = self(data, direct = 'forward',seq_len=seq_len)
        if optimizer is not None:
            optimizer.zero_grad()
            ret['loss'].backward()
            optimizer.step()

        return ret

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from data_loader.ipynb
